In [ ]:
import wave
import os
import librosa

In [ ]:
Path_to_train = "train\wav"
subfolders = os.listdir(Path_to_train)
data = []
for s in subfolders:
    files = os.listdir(Path_to_train + "/" +s)
    data.extend([Path_to_train + "/" + s+ "/" + f for f in files])
data[:10]

In [ ]:
#read text from every transcription audio
def read_text( text_path):
    text = []
    with open(text_path) as fp:
        line = fp.readline()
        while line:
        # TODO: fix spaces in in amharic text
            text.append(line)
            line = fp.readline()
    return text

In [ ]:
#extract the transcription and the label 
label=[]
transcriptions = []
for t in text:
    sp = t.split("\t")
    sp = sp.strip("\n")
    if len(sp) > 1:
        label.append(sp[0])
        transcriptions.append(sp[1])
transcriptions[:10]

In [ ]:
#get audio path , every path must corespond to transcription , get the transprion in the doc and append to audio path 
audio_path=[0]*len(transcriptions)
for d in data:
    _d = d.strip(".wav")
    sp = _d.split("/")[2]
    index = label.index(sp)
    audio_path[index] = d

In [ ]:
#calculate duration 
duration_of_recordings=[]
for d in audio_path:
    audio, fs = librosa.load(d, sr=None)
    duration_of_recordings.append(float(len(audio)/fs))

In [ ]:
import pandas as pd 
data=pd.DataFrame({'key': audio_path,'text': transcriptions, 'duration':duration_of_recordings})

# Resize and standardize
We resize all the audio samples to have the same length by either extending its duration by padding it with silence, or by truncating it.
We use the right padding method

Standardize the sample rate.


In [ ]:
# used to load audio file
#specifying sample rate will resize all the files i.e Audio will be automatically resampled to the given rate
class Loader:
  def __init__(self, sample_rate,duration,mono):
    self.sample_rate=sample_rate
    self.duration=duration
    self.mono=mono
    self.channel = 2


  def load(self,filepath):
    sig, sr = torchaudio.load(filepath)
    aud = sig, sr
    return aud

  #before using this function kindly change your file paths for it to work


  def rechannel(self, aud):    #convert mono to stereo
    # aud=self.aud
    sig, sr = aud
  

    if (sig.shape[0] == self.channel):
      # Nothing to do
      return self.aud

    if (self.channel == 1):
      # Convert from stereo to mono by selecting only the first channel
      resig = sig[:1, :]
    else:
      # Convert from mono to stereo by duplicating the first channel
      resig = torch.cat([sig, sig])

    aud = resig, sr
  def resample(self,aud):                    #standardize sample rate
    sig, sr = aud
    
    if (sr == self.sample_rate):
      # Nothing to do
      return aud

    num_channels = sig.shape[0]
    # Resample first channel
    resig = torchaudio.transforms.Resample(sr, self.sample_rate)(sig[:1,:])
    if (num_channels > 1):
      # Resample the second channel and merge both channels
      retwo = torchaudio.transforms.Resample(sr, self.sample_rate)(sig[1:,:])
      resig = torch.cat([resig, retwo])
      aud = resig, self.sample_rate
    return aud

  # ----------------------------
  # Pad (or truncate) the signal to a fixed length 'max_ms' in milliseconds
  # ----------------------------
  def pad_trunc(self,aud):
    sig, sr = aud
    num_rows, sig_len = sig.shape
    max_len = sr//1000 * self.duration

    if (sig_len > max_len):
      # Truncate the signal to the given length
      sig = sig[:,:max_len]

    elif (sig_len < max_len):
      # Length of padding to add at the beginning and end of the signal
      pad_begin_len = random.randint(0, max_len - sig_len)
      pad_end_len = max_len - sig_len - pad_begin_len

      # Pad with 0s
      pad_begin = torch.zeros((num_rows, pad_begin_len))
      pad_end = torch.zeros((num_rows, pad_end_len))

      sig = torch.cat((pad_begin, sig, pad_end), 1)
      aud = sig, sr
    return aud

In [ ]:
class PreprocessingPipeline:
    
  '''Processes audio files in a directory by applying the following steps
    1. Load the data, convert to stereo and resample sampling rate
    2. Pad the audio
  '''
    def __init__(self):
        self.padder=None
        self._loader=None
   

    def process(self,audio_files_directory):
        for root, directories, files in os.walk(audio_files_directory):
            for filename in files:
                filepath = os.path.join(root, filename)
                self._process_file(filepath)
                print(f"Processed file {filepath}")
                self._convert_mfcc(filepath)
    
    def _process_file(self,filepath):     
        signal=self.loader.load(filepath)
        signal = self.loader.make_stereo(signal)
        signal = self.loader.resample(signal)
        signal= self.loader.pad_trunc(signal)
        

 

In [ ]:
DURATION=4000
SAMPLE_RATE=44100
MONO=False

In [ ]:
loader=Loader(SAMPLE_RATE, DURATION, MONO)

In [ ]:
preprocessing_pipeline=PreprocessingPipeline()
preprocessing_pipeline.loader=loader

In [ ]:
preprocessing_pipeline.process(Path_to_train)

In [5]:
#create spectogram and covert it to mfcc then save

def gen_features(audio_files_directory, out_path):
    sr = 44100
    for root, directories, files in os.walk(audio_files_directory):
        for filename in files:
            audio_path = os.path.join(root, filename)
            audio_out = os.path.join(out_path, filename)
            audio_data, sr = librosa.load(audio_path) # getting y
            spectogram = librosa.stft(audio_data)     
            mfcc_feature_list = librosa.feature.mfcc(spectogram,sr=sr) # create mfcc features
            np.savetxt(str(audio_out), spectogram, delimiter ="\t")
            print("done saving")
                                      
a = "C:\\Users\\cthru\Documents\\psnl_projects\\10academy\\Swahili-NLP\\data\\train\\wav"
b = "C:\\Users\\cthru\\Documents\\psnl_projects\\10academy\\Swahili-NLP\\data\\features"
        
gen_features(a,b) 

done saving
